In [1]:
from ift6758.data.acquisition import NHLGameData

In [2]:
data_path_raw = './../ift6758/data/json_raw/'
nhl_games_data = NHLGameData(data_path_raw)

In [3]:
for year in range(2016,2021):
    nhl_games_data.fetch_season(year)

Loading from cache file ./../ift6758/data/json_raw/2016/2016-regular.pkl
Found 1230 regular games for season 2016-2017
Loading from cache file ./../ift6758/data/json_raw/2016/2016-playoff.pkl
Found 102 playoff games for season 2016-2017
Loading from cache file ./../ift6758/data/json_raw/2017/2017-regular.pkl
Found 1271 regular games for season 2017-2018
Loading from cache file ./../ift6758/data/json_raw/2017/2017-playoff.pkl
Found 105 playoff games for season 2017-2018
Loading from cache file ./../ift6758/data/json_raw/2018/2018-regular.pkl
Found 1271 regular games for season 2018-2019
Loading from cache file ./../ift6758/data/json_raw/2018/2018-playoff.pkl
Found 105 playoff games for season 2018-2019
Loading from cache file ./../ift6758/data/json_raw/2019/2019-regular.pkl
Found 1271 regular games for season 2019-2020
Loading from cache file ./../ift6758/data/json_raw/2019/2019-playoff.pkl
Found 105 playoff games for season 2019-2020
Loading from cache file ./../ift6758/data/json_raw/2

In [4]:
from ift6758.data.cleaning import DataCleaner

In [5]:
data_path_clean = './../ift6758/data/json_clean/'
data_cleaner = DataCleaner(data_raw=nhl_games_data, data_path_clean=data_path_clean)

In [6]:
for year in range(2016,2021):
    data_cleaner.clean_season(year)

### Test extract

In [7]:
import pandas as pd

# print out a pandas head of the data
data_2016 = pd.read_pickle(data_path_clean + '2016/2016.pkl')
data_2017 = pd.read_pickle(data_path_clean + '2017/2017.pkl')
data_2018 = pd.read_pickle(data_path_clean + '2018/2018.pkl')
data_2019 = pd.read_pickle(data_path_clean + '2019/2019.pkl')
data_2020 = pd.read_pickle(data_path_clean + '2020/2020.pkl')
    
data_2016.sample(10)

,game_id,time,event_type,period,team,coordinates,x,y,shooter,goalie,shot_type,empty_net,strength,opposite_team_side
70453,2016021167,13:32,SHOT,1,Carolina Hurricanes,"{'x': -37.0, 'y': 17.0}",-37.0,17.0,Lucas Wallmark,Matt Murray,Wrist Shot,False,None,left
18012,2016020302,01:10,SHOT,3,Philadelphia Flyers,"{'x': -31.0, 'y': -7.0}",-31.0,-7.0,Jakub Voracek,Henrik Lundqvist,Snap Shot,False,None,left
74764,2016030123,00:13,SHOT,4,Boston Bruins,"{'x': -59.0, 'y': -12.0}",-59.0,-12.0,Brad Marchand,Craig Anderson,Snap Shot,False,None,left
29035,2016020484,12:27,SHOT,1,Columbus Blue Jackets,"{'x': 21.0, 'y': 7.0}",21.0,7.0,Brandon Dubinsky,Peter Budaj,Snap Shot,False,None,right
12205,2016020205,14:24,SHOT,2,Winnipeg Jets,"{'x': 10.0, 'y': -29.0}",10.0,-29.0,Tyler Myers,Louis Domingue,Slap Shot,False,None,right
45831,2016020763,17:57,SHOT,2,Dallas Stars,"{'x': 43.0, 'y': -1.0}",43.0,-1.0,John Klingberg,Ondrej Pavelec,Wrist Shot,False,None,right
32373,2016020539,15:18,SHOT,1,Winnipeg Jets,"{'x': -69.0, 'y': 40.0}",-69.0,40.0,Blake Wheeler,Sergei Bobrovsky,Wrist Shot,False,None,left
16427,2016020276,03:30,SHOT,1,Calgary Flames,"{'x': -69.0, 'y': 13.0}",-69.0,13.0,Sam Bennett,Robin Lehner,Backhand,False,None,left
71517,2016021184,18:25,SHOT,3,St. Louis Blues,"{'x': 39.0, 'y': -37.0}",39.0,-37.0,Alexander Steen,Michael Hutchinson,Slap Shot,False,None,right
23090,2016020384,12:22,SHOT,3,New York Islanders,"{'x': 62.0, 'y': 20.0}",62.0,20.0,Nick Leddy,Henrik Lundqvist,Snap Shot,False,None,right


In [8]:
print(data_2016.shape)
print(data_2017.shape)
print(data_2018.shape)
print(data_2019.shape)
print(data_2020.shape)

(79656, 14)
(83098, 14)
(82216, 14)
(70116, 14)
(54954, 14)
